In [9]:
#Importing packages and functions
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [1]:
#Getting the dataset
!wget --no-check-certificate \
    -O /tmp/sentiment.csv https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P

--2020-10-27 17:53:04--  https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving drive.google.com (drive.google.com)... 74.125.124.101, 74.125.124.138, 74.125.124.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.124.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/p8aieq2rp63lhgtqplfppmseafvs90gf/1603821150000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P [following]
--2020-10-27 17:53:04--  https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/p8aieq2rp63lhgtqplfppmseafvs90gf/1603821150000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)... 64.233.183.132, 2607:f8b0:4001:c0b::84
Connecting to doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)|64.233

In [5]:
import numpy as np
import pandas as pd

dataset= pd.read_csv('/tmp/sentiment.csv')

sentences= dataset['text'].tolist()
labels= dataset['sentiment'].tolist()

training_size= int(len(sentences)*0.8)

training_set= sentences[0:training_size]
training_labels=labels[0:training_size]

testing_set=sentences[training_size:]
testing_labels=labels[training_size:]

training_labels=np.array(training_labels)
testing_labels=np.array(testing_labels)

In [14]:
#Tokenizing Sets
vocab_size = 1000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"


 
tokenizer= Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_set)
tokenizer.word_index


train_sequences= tokenizer.texts_to_sequences(training_set)
train_padded= pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating= trunc_type)

test_sequences= tokenizer.texts_to_sequences(testing_set)
test_padded= pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating= trunc_type)

In [15]:
#Model Building
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 16)           16000     
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 9606      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 25,613
Trainable params: 25,613
Non-trainable params: 0
_________________________________________________________________


In [16]:
num_epochs = 10
model.fit(train_padded, training_labels, epochs=num_epochs, validation_data=(test_padded, testing_labels))

Epoch 1/10
50/50 [==============================] - 0s 5ms/step - loss: 0.6939 - accuracy: 0.5060 - val_loss: 0.6941 - val_accuracy: 0.4110
Epoch 2/10
50/50 [==============================] - 0s 2ms/step - loss: 0.6928 - accuracy: 0.5229 - val_loss: 0.6951 - val_accuracy: 0.4110
Epoch 3/10
50/50 [==============================] - 0s 2ms/step - loss: 0.6926 - accuracy: 0.5229 - val_loss: 0.6957 - val_accuracy: 0.4110
Epoch 4/10
50/50 [==============================] - 0s 2ms/step - loss: 0.6924 - accuracy: 0.5229 - val_loss: 0.6966 - val_accuracy: 0.4110
Epoch 5/10
50/50 [==============================] - 0s 2ms/step - loss: 0.6923 - accuracy: 0.5229 - val_loss: 0.6970 - val_accuracy: 0.4110
Epoch 6/10
50/50 [==============================] - 0s 2ms/step - loss: 0.6921 - accuracy: 0.5229 - val_loss: 0.6980 - val_accuracy: 0.4110
Epoch 7/10
50/50 [==============================] - 0s 2ms/step - loss: 0.6919 - accuracy: 0.5229 - val_loss: 0.6986 - val_accuracy: 0.4110
Epoch 8/10
50/50 [==

In [ ]:
#Predicting on new data
fake_reviews = ['I love this phone', 'I hate spaghetti', 
                'Everything was cold',
                'Everything was hot exactly as I wanted', 
                'Everything was green', 
                'the host seated us immediately',
                'they gave us free chocolate cake', 
                'not sure about the wilted flowers on the table',
                'only works when I stand on tippy toes', 
                'does not work when I stand on my head']

print(fake_reviews) 

padding_type= 'post'
sample_sequences=tokenizer.texts_to_sequences(fake_reviews)
sample_padded= pad_sequences(sample_sequences, maxlen= max_length, padding=padding_type)

classes= model.predict(sample_padded)

for x in range(len(fake_reviews)):
  print(fake_reviews[x])
  print(classes[x])
  print('\n')

